### Gender prediction with classification method
For the model_gender_classification, we've tailored a convolutional neural network (CNN) to distinguish between male and female subjects. This model leverages a custom architecture, beginning with a base of pretrained layers for feature extraction, followed by additional dense layers for refined learning. These layers collectively enhance the model's ability to accurately classify gender based on facial features. The use of a data generator here, as with age regression, ensures efficient handling of large datasets, allowing for dynamic data augmentation and streamlined model training without excessive memory consumption.

In [1]:
import os
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
%matplotlib inline
import matplotlib.pyplot as plt
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from keras.optimizers import Adam
from tensorflow.keras.layers import SpatialDropout2D, SeparableConv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

For managing large datasets efficiently and minimizing memory usage, we utilize data generators. These generators stream data in batches directly to the model during training, enabling real-time data augmentation and improving model performance without overwhelming system resources.

In [2]:
#data generator method
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, list_IDs, load_from, batch_size=32, dim=(224,224), n_channels=3,
                 shuffle=True, preprocessing=None):
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.load_from = load_from
        self.preprocessing = preprocessing
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        # Generate one batch of data
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        # Updates indexes after each epoch
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # Generates data containing batch_size samples
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img_path = os.path.join(self.load_from, ID)
            img = load_img(img_path, target_size=self.dim)  # This is a PIL image
            img = img_to_array(img)  # Convert to numpy array
            img = self.preprocessing(img) if self.preprocessing else img  # Apply preprocessing
            X[i,] = img

            label = ID.split('_')[1]
            y[i] = int(label)

        return X, y


In [3]:
#data gen config
load_from_train = 'data/train/'
load_from_val = 'data/test/'

def preprocess_input_vggface(x):
    x = np.array(x, dtype=np.float32)
    x[..., 0] -= 93.5940  # Subtract the mean red value
    x[..., 1] -= 104.7624 # Subtract the mean green value
    x[..., 2] -= 129.1863 # Subtract the mean blue value
    return x

# Parameters
batch_size = 16
params = {'dim': (224, 224),
          'batch_size': batch_size,
          'n_channels': 3,
          'shuffle': True,
          'preprocessing': preprocess_input_vggface}

ids_train = os.listdir(load_from_train) # IDs for training
ids_val = os.listdir(load_from_val) # IDs for validation

training_generator = DataGenerator(list_IDs = ids_train, load_from = load_from_train, **params)
validation_generator = DataGenerator(list_IDs = ids_val, load_from = load_from_val, **params)

### Build the model with VGGFace

In [8]:
# Load VGGFace model
# Creating a base model with SENet50
base_model = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output    
# First block
x = BatchNormalization()(x)
x = SpatialDropout2D(0.5)(x)
x = SeparableConv2D(512, (3, 3), padding='same', activation='relu')(x)
# Removed MaxPooling2D layer here to prevent dimensionality issues

# Second block, adjust similarly as needed
x = BatchNormalization()(x)
x = SpatialDropout2D(0.5)(x)
x = SeparableConv2D(512, (3, 3), padding='same', activation='relu')(x)


# Fully connected system
x = Flatten()(x)
x = BatchNormalization()(x)
x = AlphaDropout(0.5)(x)  # Make sure to call AlphaDropout on `x`
x = Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
# Output layer
predictions = Dense(1, activation='sigmoid')(x)  # Make sure to call this layer on `x`

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Adjust the learning rate
optimizer = Adam(learning_rate=0.0001)  # Lower learning rate

# Compile the model with the new optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
#lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
os.makedirs('saved_models0', exist_ok=True)
filename = 'saved_models0/-{epoch:02d}---{val_loss:.4f}-{val_accuracy:.4f}---{loss:.4f}-{accuracy:.4f}.h5'

checkpoint = ModelCheckpoint(filename, monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_accuracy', factor=0.8, patience=4, verbose=1, mode='auto', min_delta=0.0001, min_lr=0.0001)
callbacks_list = [checkpoint, reduceLROnPlat]

In [10]:
# Fit the model using the Dataset
history = model.fit(
    training_generator,
    validation_data=validation_generator,
    epochs=30,
    callbacks = callbacks_list
)

Epoch 1/30
1185/1185 [==============================] - ETA: 0s - loss: 0.3716 - accuracy: 0.8326
Epoch 1: val_accuracy improved from -inf to 0.92990, saving model to saved_models0\-01---0.2235-0.9299---0.3716-0.8326.h5
1185/1185 [==============================] - 363s 298ms/step - loss: 0.3716 - accuracy: 0.8326 - val_loss: 0.2235 - val_accuracy: 0.9299 - lr: 1.0000e-04
Epoch 2/30
 965/1185 [=======================>......] - ETA: 54s - loss: 0.2437 - accuracy: 0.9051